## Here's a basic PyTorch pipeline where:

***YOLOv5 is used to detect faces or persons (driver region),***

***ResNet-50 is used to extract features from the detected region,***

***You can then feed the ResNet features into your downstream model (e.g., LSTM for behavior prediction).***



#Step 1: Install YOLOv5 + Requirements



In [6]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17326 (from 3)
Receiving objects: 100% (17410/17410), 16.30 MiB | 18.26 MiB/s, done.
Resolving deltas: 100% (11934/11934), done.
/content/yolov5/yolov5


## Step 2: Load YOLOv5 + ResNet-50 Models

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import cv2
import numpy as np

# Load YOLOv5 model (YOLOv5s by default)
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Load ResNet-50 pretrained
resnet = models.resnet50(pretrained=True)
resnet.eval()
resnet_feat = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove last FC layer

# Image transform for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-4-23 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 187MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 134MB/s]


## Step 3: Upload Image

In [4]:
from google.colab import files
uploaded = files.upload()

# Read the uploaded image
image_path = list(uploaded.keys())[0]
img = cv2.imread(image_path)


Saving Chaitanya.png to Chaitanya.png


## Step 4: Detect with YOLO and Extract Features via ResNet-50

In [5]:
# Run YOLO inference
results = yolo(img)
detections = results.pandas().xyxy[0]

for idx, det in detections.iterrows():
    if det['name'] == 'person':  # or 'face' if using fine-tuned YOLO
        x1, y1, x2, y2 = map(int, [det['xmin'], det['ymin'], det['xmax'], det['ymax']])
        cropped_img = img[y1:y2, x1:x2]

        # Convert to PIL and preprocess
        pil_img = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
        input_tensor = transform(pil_img).unsqueeze(0)

        # Get features
        with torch.no_grad():
            features = resnet_feat(input_tensor).squeeze().numpy()
            print(f"Feature vector shape: {features.shape}")


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Feature vector shape: (2048,)


## Step 5: Detect multiple states

In [7]:
import torch.nn as nn

class MultiClassDriverBehaviorLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MultiClassDriverBehaviorLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        logits = self.fc(lstm_out[:, -1, :])  # last time-step output
        return logits


## Step 6: Output

In [9]:
# Dummy input (sequence of ResNet-50 features)
features_seq = torch.tensor([features]).unsqueeze(0)  # shape: [1, 1, 2048]

# Model
num_classes = 5
model = MultiClassDriverBehaviorLSTM(input_size=2048, hidden_size=128, num_classes=num_classes)
model.eval()

# Predict
with torch.no_grad():
    output_logits = model(features_seq)
    predicted_class = torch.argmax(output_logits, dim=1).item()

    class_labels = ['Focused', 'Yawning', 'Distracted', 'Angry']
    print(f"Predicted Behavior: {class_labels[predicted_class]}")



Predicted Behavior: Distracted


### Driver Drowsy Detection

In [13]:
###Driver Drowsy Detection

In [11]:
import torch.nn as nn

# Example LSTM Model for Sequence Prediction
class DriverBehaviorLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DriverBehaviorLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # lstm_out has shape [batch_size, seq_len, hidden_size]
        output = self.fc(lstm_out[:, -1, :])  # Take output of last time step
        return output

# Assuming you have a sequence of feature vectors (features over time)
features_seq = torch.tensor([features])  # Example: features from multiple frames
features_seq = features_seq.unsqueeze(0)  # Adding batch dimension

# Model instantiation
lstm_model = DriverBehaviorLSTM(input_size=2048, hidden_size=128, output_size=1)  # Output: binary classification (0 or 1)

# Prediction (for example, predicting drowsiness)
prediction = lstm_model(features_seq)
print(f"Driver Behavior Prediction: {prediction}")


Driver Behavior Prediction: tensor([[-0.14744]], grad_fn=<AddmmBackward0>)


In [12]:
import torch.nn.functional as F

# Apply sigmoid to the raw prediction
probability = torch.sigmoid(prediction)
print(f"Prediction Probability: {probability.item()}")

# Threshold the probability (e.g., 0.5 for binary classification)
if probability.item() > 0.5:
    print("Driver is Drowsy")
else:
    print("Driver is Not Drowsy")


Prediction Probability: 0.46320727467536926
Driver is Not Drowsy
